#SWIN Transformer


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 133kB 72.8MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 163kB 57.3MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=30123be72368c2b2bddb0dc65aabf42bff565b7ddcc5cf9feaee1031b3b754ea
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0fdbb4f3dfdef226fe728d8c6f0c82bb4aa116515244f0e74fecba06851263b7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [5]:
from model.swin import SwinTransformer
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [25]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 64,
    "hidden_dim" : 96,
    "channels" : 3,
    "num_classes" : 10,
    "head_dim" : 32,
    "window_size" : 4,
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='Swin')

In [10]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [26]:
model = SwinTransformer(
    hidden_dim= config['hidden_dim'],
    layers=(2, 2, 6, 2),
    heads=(3, 6, 12, 24),
    channels=config['channels'],
    num_classes=config['num_classes'],
    head_dim=config['head_dim'],
    window_size=config['window_size'],
    downscaling_factors=(2, 2, 2, 2),
    relative_pos_embedding=True).to(device)

In [27]:
from torchsummary import summary
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Unfold-1             [-1, 12, 1024]               0
            Linear-2           [-1, 32, 32, 96]           1,248
      PatchMerging-3           [-1, 32, 32, 96]               0
         LayerNorm-4           [-1, 32, 32, 96]             192
            Linear-5          [-1, 32, 32, 288]          27,648
            Linear-6           [-1, 32, 32, 96]           9,312
   WindowAttention-7           [-1, 32, 32, 96]               0
           PreNorm-8           [-1, 32, 32, 96]               0
          Residual-9           [-1, 32, 32, 96]               0
        LayerNorm-10           [-1, 32, 32, 96]             192
           Linear-11          [-1, 32, 32, 384]          37,248
             GELU-12          [-1, 32, 32, 384]               0
           Linear-13           [-1, 32, 32, 96]          36,960
      FeedForward-14           [-1, 32,

In [28]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [29]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 1.9646 - acc: 0.2677 - val_loss : 1.8323 - val_acc: 0.3161




Epoch : 2 - loss : 1.6838 - acc: 0.3827 - val_loss : 1.6025 - val_acc: 0.4176




Epoch : 3 - loss : 1.5398 - acc: 0.4426 - val_loss : 1.5120 - val_acc: 0.4561




Epoch : 4 - loss : 1.4494 - acc: 0.4796 - val_loss : 1.4169 - val_acc: 0.4905




Epoch : 5 - loss : 1.3717 - acc: 0.5069 - val_loss : 1.3482 - val_acc: 0.5172



...


Epoch : 16 - loss : 0.9909 - acc: 0.6495 - val_loss : 1.0118 - val_acc: 0.6475




Epoch : 17 - loss : 0.9558 - acc: 0.6638 - val_loss : 1.0229 - val_acc: 0.6395




Epoch : 18 - loss : 0.9403 - acc: 0.6685 - val_loss : 0.9810 - val_acc: 0.6602




Epoch : 19 - loss : 0.9228 - acc: 0.6737 - val_loss : 0.9880 - val_acc: 0.6520




Epoch : 20 - loss : 0.8999 - acc: 0.6831 - val_loss : 0.9578 - val_acc: 0.6609



In [30]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May  9 08:19:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    50W / 250W |   3983MiB / 16280MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------